# A simple chatbot agent

## Prepare the environment

In [1]:
import sys
sys.path.append('../../..')

from autogen import ConversableAgent, register_function

# from inswitch.llm.model import get_openai_model_config
from inswitch.agent.basic import get_chat_agent, get_fixed_reply_agent
from inswitch.agent.apiagent import ApiAgent
from inswitch.util import second_last_msg

from usecases.fill.filluc.mockupnerv.session import make_request

## The simplified knowledge-providing agent

In [2]:
with open("../data/mockup_single_machine_tripple.txt", "r") as api_doc:
    content = api_doc.read()

knowledge_provider = get_fixed_reply_agent(
    'knowledge_provider',
    reply=content
)

In [3]:
decision_maker_message = '''You are the decision maker.
You get the user's intents, together with some knowledge about 
what machines, services, workloads, version etc., are avaliable in the system.
Based on these, you decide which workloads, and their versions. should be deployed on which machine.
Remember to always and only use the concrete version numbers, not the version names.
Reply with the deployment plan, i.e., machine serial number, workload names, and their version numbers. With label and values'''

decision_maker = get_chat_agent(
    "decision_maker",
    system_message=decision_maker_message
)

## The simplied doc providing agent: no RAG in it - only manually selected text

In [4]:
with open("../data/nerve_api_dna.txt", "r") as api_doc:
    content = api_doc.read()

api_doc_provider = get_fixed_reply_agent(
    'api_doc_provider', 
    reply=content
)

moderator = get_fixed_reply_agent(
    name="moderator",
    reply = ""
)


## Here are the API invocation agents

In [5]:
# nerv_tool_driver_system_message = '''You are a helpful assistent.
# You have access to a tool to call the Nerve DNA API to fulfil the user's intent.
# You will get from the context a document of the Nerve DNA API.
# Using the document, you will figure out how to call the API, i.e., using what endpoint and method.
# You will also need to generate the configuration file as the data of the api call, to fulfil the task assigned to you.
# '''

# nerv_tool_driver = get_chat_agent(
#     "nerv_tool_driver",
#     system_message = nerv_tool_driver_system_message
# )

# nerv_tool_executor = get_fixed_reply_agent(
#     "nerv_tool_executor",
#     reply = ""
# )

# register_function(
#     make_request,
#     caller = nerv_tool_driver,
#     executor = nerv_tool_executor,
#     description = "This is the functionto send a request to the Nerve API for deploying workloads etc. "
# )

# nerv_tool_executor.register_nested_chats(
#     [
#         {
#             "recipient": nerv_tool_driver,
#             "max_turns": 2,
#             "summary_method": second_last_msg
#         }
#     ],
#     trigger = lambda sender: sender not in [nerv_tool_driver]
# )

nerv_api_system_message = '''
You have access to a tool to call the Nerve DNA API to fulfil the user's intent.
The task is in the context.
You will get from the context a document of the Nerve DNA API.
Using the document, you will figure out how to call the API, i.e., using what endpoint and method.
You will also need to generate the configuration file as the data of the api call, to fulfil the task assigned to you.
If you don't have the version for some workloads, 
you need to call the API to get the available version numbers and their hash code. Use the first version by default 
If you don't have the hash code, call the API using the workload and version to get its unique information, which include hash code.
'''

nerve_api_agent = ApiAgent(
    "nerve_api_agent", 
    system_message=nerv_api_system_message, 
    max_internal_turns=6
)


nerve_api_agent.register_api_function(
    make_request,
    description = "This is the functionto send a request to the Nerve API for deploying workloads etc. "
)


The following parameters of the function 'make_request' with default values are not annotated: 'files', 'workaround'.


## Here we go!
A simple sequence: user_intent -> api_doc_provider -> nerv_tool_executor

Current intent is hard coded.

In [6]:
user_intent = '''I am Customer1. 
I want to add a new service to my machine to record all the alarms
'''

intent_provider = get_fixed_reply_agent(
    'intent_provier',
    reply = user_intent
)

decision_chat_results = moderator.initiate_chats(
    [
        {
            "recipient": intent_provider,
            "message": "what do you want?",
            "max_turns": 1,
            "summary_method": "last_msg"
        },
        {
            "recipient": knowledge_provider,
            "message": "What do you know about the system",
            "max_turns": 1,
            "summary_method": "last_msg"
        },
        {
            "recipient": decision_maker,
            "message": "What should be deployed",
            "max_turns": 1,
            "summary_method": "last_msg"
        }
    ]
)

deploy_plan = decision_chat_results[-1].summary

print(deploy_plan)

deployment_plan_announcer = get_fixed_reply_agent(
    'deployment_plan_announcer',
    reply=deploy_plan
)

chat_result = moderator.initiate_chats(
    [
        {
            "recipient": deployment_plan_announcer,
            "message": "What should be deployed? on which machine?",
            "max_turns": 1,
            "summary_method": "last_msg",
        },
        {
            "recipient": api_doc_provider,
            "message": "Please provide me the api doc",
            "max_turns": 1,
            "summary_method": "last_msg",
        },
        {
            "recipient": nerve_api_agent,
            "message": "you can see the intent and the knowledge in the context",
            "max_turns": 1,
            "summary_method": "last_msg"
        }
    ]
)



********************************************************************************
Starting a new chat....

********************************************************************************
moderator (to intent_provier):

what do you want?

--------------------------------------------------------------------------------
intent_provier (to moderator):

I am Customer1. 
I want to add a new service to my machine to record all the alarms


--------------------------------------------------------------------------------

********************************************************************************
Starting a new chat....

********************************************************************************
moderator (to knowledge_provider):

What do you know about the system
Context: 
I am Customer1. 
I want to add a new service to my machine to record all the alarms


--------------------------------------------------------------------------------
knowledge_provider (to moderator):

Customer1 

## Next steps:

1. Replace current api_doc_provider (a fixed_reply_agent) by a RAG assistant, that extracts from a complete API document the parts that are relevant to DNA and deployment.
2. Add in the beginning of the sequence another agent (or agents) to extract the list of workloads. Eventually, the input intent should be like "I want to be able to monitor the thermal stability of machine M0001"